# 🔤 Tokenization in Hugging Face

---

## 📚 What You'll Learn

Tokenization is the **foundational step** in Natural Language Processing (NLP) that converts human-readable text into a format that machine learning models can understand. In this notebook, you'll learn:

1. **What is Tokenization?** - Breaking text into smaller, meaningful units
2. **Types of Tokenization** - Word-level, Subword-level, and Character-level
3. **Hugging Face Tokenizers** - Using AutoTokenizer for various models
4. **Understanding Token Components** - input_ids, attention_mask, token_type_ids
5. **Practical Dataset Tokenization** - Preparing data for model training

---

## 🎯 Why Tokenization Matters

| Purpose | Description |
|---------|-------------|
| **Text Preprocessing** | Simplifies handling of punctuation, casing, and special characters |
| **Numerical Representation** | Converts text to token IDs for model consumption |
| **Memory Efficiency** | Subword tokenization reduces vocabulary size while maintaining coverage |
| **Foundation for NLP Tasks** | Essential for NER, translation, summarization, and more |

## 🛠️ Setup and Installation

In [1]:
# Install required libraries (uncomment if needed)
# !pip install transformers datasets torch -q

In [2]:
# Import essential libraries
from transformers import AutoTokenizer
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


---

## 📖 Part 1: Types of Tokenization Methods

There are three main approaches to breaking text into tokens, each with its own strengths and use cases.

### 1️⃣ Word-Level Tokenization

The simplest approach - splits text by whitespace and punctuation into complete words.

**Pros:**
- Simple and intuitive
- Each token represents a complete word

**Cons:**
- Struggles with out-of-vocabulary (OOV) words
- Requires large vocabulary for diverse languages
- Doesn't capture word morphology (prefixes, suffixes)

**Models using this:** Word2Vec, GloVe

In [3]:
# ✨ Example: Simple Word-Level Tokenization

def simple_word_tokenize(text):
    """Basic word-level tokenization using split."""
    return text.split()

# Let's try it with a tech-related sentence
sample_text = "Artificial Intelligence is transforming healthcare and education"
word_tokens = simple_word_tokenize(sample_text)

print(f"📝 Original Text: '{sample_text}'")
print(f"🔢 Number of Tokens: {len(word_tokens)}")
print(f"📋 Tokens: {word_tokens}")

📝 Original Text: 'Artificial Intelligence is transforming healthcare and education'
🔢 Number of Tokens: 7
📋 Tokens: ['Artificial', 'Intelligence', 'is', 'transforming', 'healthcare', 'and', 'education']


In [4]:
# 🚫 Problem with Word-Level: Out-of-Vocabulary Words

# Imagine a fixed vocabulary
vocabulary = {"artificial", "intelligence", "is", "good", "for", "society"}

text = "Artificial Intelligence is revolutionizing biotechnology"
tokens = simple_word_tokenize(text.lower())

print("🔍 Checking tokens against vocabulary:")
for token in tokens:
    status = "✅ Known" if token in vocabulary else "❌ OOV (Unknown)"
    print(f"   '{token}': {status}")

print("\n💡 Words like 'revolutionizing' and 'biotechnology' would be unknown!")

🔍 Checking tokens against vocabulary:
   'artificial': ✅ Known
   'intelligence': ✅ Known
   'is': ✅ Known
   'revolutionizing': ❌ OOV (Unknown)
   'biotechnology': ❌ OOV (Unknown)

💡 Words like 'revolutionizing' and 'biotechnology' would be unknown!


### 2️⃣ Subword-Level Tokenization

The **gold standard** for modern NLP! Breaks words into smaller meaningful units.

**Pros:**
- ✅ Handles OOV words by breaking them into known subwords
- ✅ Captures morphology (prefixes, suffixes, roots)
- ✅ Compact vocabulary with wide coverage
- ✅ Works well across languages

**Algorithms:**
- **BPE (Byte-Pair Encoding)**: Used by GPT-2, GPT-3, RoBERTa
- **WordPiece**: Used by BERT, DistilBERT
- **SentencePiece**: Used by T5, ALBERT, XLNet

**Models using this:** BERT, GPT, T5, RoBERTa, and most modern transformers

In [5]:
# ✨ Example: Subword Tokenization with BERT

# Load BERT's WordPiece tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Let's see how it handles complex words
complex_words = [
    "cryptocurrency",
    "unbelievable",
    "internationalization",
    "neuroscientist"
]

print("🔬 Subword Tokenization Examples:")
print("=" * 50)

for word in complex_words:
    tokens = bert_tokenizer.tokenize(word)
    print(f"\n📝 Word: '{word}'")
    print(f"   Subwords: {tokens}")
    print(f"   Count: {len(tokens)} tokens")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

🔬 Subword Tokenization Examples:

📝 Word: 'cryptocurrency'
   Subwords: ['crypt', '##oc', '##ur', '##ren', '##cy']
   Count: 5 tokens

📝 Word: 'unbelievable'
   Subwords: ['unbelievable']
   Count: 1 tokens

📝 Word: 'internationalization'
   Subwords: ['international', '##ization']
   Count: 2 tokens

📝 Word: 'neuroscientist'
   Subwords: ['ne', '##uro', '##sc', '##ient', '##ist']
   Count: 5 tokens


In [6]:
# 💡 Understanding the '##' Symbol in WordPiece

text = "The microprocessor revolutionized computing"
tokens = bert_tokenizer.tokenize(text)

print(f"📝 Original: '{text}'")
print(f"\n🔍 Tokens: {tokens}")

print("\n📚 Explanation:")
print("   • '##' prefix indicates the token is a CONTINUATION of the previous word")
print("   • Tokens without '##' are either standalone words or word beginnings")
print("   • This helps the model understand word boundaries")

📝 Original: 'The microprocessor revolutionized computing'

🔍 Tokens: ['the', 'micro', '##pro', '##ces', '##sor', 'revolution', '##ized', 'computing']

📚 Explanation:
   • '##' prefix indicates the token is a CONTINUATION of the previous word
   • Tokens without '##' are either standalone words or word beginnings
   • This helps the model understand word boundaries


### 3️⃣ Character-Level Tokenization

Breaks text into individual characters. Especially useful for:
- Languages without clear word boundaries (Chinese, Japanese, Korean)
- Handling typos and misspellings
- Very fine-grained text analysis

**Pros:**
- No OOV problem (limited character set)
- Perfect for logographic languages

**Cons:**
- Very long sequences
- Harder for models to learn word meanings

In [7]:
# ✨ Example: Character-Level Tokenization

def char_tokenize(text):
    """Character-level tokenization."""
    return list(text)

# English example
english_text = "Hello AI"
english_chars = char_tokenize(english_text)

print("🔤 Character-Level Tokenization:")
print(f"\n📝 English: '{english_text}'")
print(f"   Characters: {english_chars}")
print(f"   Count: {len(english_chars)} characters")

# Chinese example (if your system supports it)
chinese_text = "人工智能"  # "Artificial Intelligence" in Chinese
chinese_chars = char_tokenize(chinese_text)

print(f"\n📝 Chinese: '{chinese_text}'")
print(f"   Characters: {chinese_chars}")
print(f"   Count: {len(chinese_chars)} characters")
print("\n💡 Each Chinese character is a meaningful unit!")

🔤 Character-Level Tokenization:

📝 English: 'Hello AI'
   Characters: ['H', 'e', 'l', 'l', 'o', ' ', 'A', 'I']
   Count: 8 characters

📝 Chinese: '人工智能'
   Characters: ['人', '工', '智', '能']
   Count: 4 characters

💡 Each Chinese character is a meaningful unit!


### 📊 Comparison Summary

| Method | Vocabulary Size | OOV Handling | Sequence Length | Best For |
|--------|----------------|--------------|-----------------|----------|
| Word-Level | Very Large | Poor | Short | Simple tasks, older models |
| **Subword** | Medium | **Excellent** | Medium | **Modern Transformers** |
| Character | Small | None (No OOV) | Very Long | CJK languages, typo handling |

---

## 📖 Part 2: Hugging Face AutoTokenizer

The `AutoTokenizer` class automatically loads the correct tokenizer for any pre-trained model. This is the **recommended way** to work with tokenizers in Hugging Face.

In [11]:
# 🔄 Loading Different Tokenizers

# BERT tokenizer (WordPiece)
bert_tok = AutoTokenizer.from_pretrained('bert-base-uncased')

# GPT-2 tokenizer (BPE)
gpt2_tok = AutoTokenizer.from_pretrained('gpt2')

print("✅ Loaded tokenizers:")
print(f"   • BERT: {type(bert_tok).__name__}")
print(f"   • GPT-2: {type(gpt2_tok).__name__}")

✅ Loaded tokenizers:
   • BERT: BertTokenizerFast
   • GPT-2: GPT2TokenizerFast


In [13]:
# 🔍 Comparing How Different Tokenizers Handle the Same Text

sample = "Machine learning enables computers to learn from data"

print(f"📝 Input: '{sample}'\n")
print("🔬 Tokenization Comparison:")
print("=" * 60)

# BERT
bert_tokens = bert_tok.tokenize(sample)
print(f"\n🤖 BERT (WordPiece):")
print(f"   {bert_tokens}")
print(f"   Token count: {len(bert_tokens)}")

# GPT-2
gpt2_tokens = gpt2_tok.tokenize(sample)
print(f"\n🤖 GPT-2 (BPE):")
print(f"   {gpt2_tokens}")
print(f"   Token count: {len(gpt2_tokens)}")

📝 Input: 'Machine learning enables computers to learn from data'

🔬 Tokenization Comparison:

🤖 BERT (WordPiece):
   ['machine', 'learning', 'enables', 'computers', 'to', 'learn', 'from', 'data']
   Token count: 8

🤖 GPT-2 (BPE):
   ['Machine', 'Ġlearning', 'Ġenables', 'Ġcomputers', 'Ġto', 'Ġlearn', 'Ġfrom', 'Ġdata']
   Token count: 8


---

## 📖 Part 3: Understanding Tokenizer Outputs

When you tokenize text with Hugging Face, you get three important components:

1. **`input_ids`** - Numerical IDs representing each token
2. **`attention_mask`** - Binary mask showing real tokens (1) vs padding (0)
3. **`token_type_ids`** - Segment IDs for sentence pairs (for BERT-like models)

In [14]:
# 📊 Full Tokenization Example

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

text = "Deep learning powers modern AI applications"

# Full tokenization with all outputs
encoded = tokenizer(
    text,
    padding='max_length',    # Pad to max_length
    max_length=15,           # Maximum sequence length
    truncation=True,         # Truncate if too long
    return_tensors=None      # Return Python lists
)

print(f"📝 Original Text: '{text}'")
print("\n" + "=" * 60)
print("\n📦 Tokenizer Output:")
print(f"\n1️⃣ input_ids (Token IDs):")
print(f"   {encoded['input_ids']}")
print(f"\n2️⃣ attention_mask:")
print(f"   {encoded['attention_mask']}")
print(f"\n3️⃣ token_type_ids:")
print(f"   {encoded['token_type_ids']}")

📝 Original Text: 'Deep learning powers modern AI applications'


📦 Tokenizer Output:

1️⃣ input_ids (Token IDs):
   [101, 2784, 4083, 4204, 2715, 9932, 5097, 102, 0, 0, 0, 0, 0, 0, 0]

2️⃣ attention_mask:
   [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]

3️⃣ token_type_ids:
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [15]:
# 🔍 Decoding: Converting Token IDs Back to Text

# Get the tokens from IDs
tokens = tokenizer.convert_ids_to_tokens(encoded['input_ids'])

print("🔄 Token ID to Token Mapping:\n")
print(f"{'Token ID':<12} {'Token':<15} {'Attention':<12} {'Description'}")
print("-" * 60)

for token_id, token, attn in zip(encoded['input_ids'], tokens, encoded['attention_mask']):
    if token == '[CLS]':
        desc = "← Start token"
    elif token == '[SEP]':
        desc = "← Separator/End token"
    elif token == '[PAD]':
        desc = "← Padding token"
    elif token.startswith('##'):
        desc = "← Subword continuation"
    else:
        desc = ""
    
    print(f"{token_id:<12} {token:<15} {attn:<12} {desc}")

🔄 Token ID to Token Mapping:

Token ID     Token           Attention    Description
------------------------------------------------------------
101          [CLS]           1            ← Start token
2784         deep            1            
4083         learning        1            
4204         powers          1            
2715         modern          1            
9932         ai              1            
5097         applications    1            
102          [SEP]           1            ← Separator/End token
0            [PAD]           0            ← Padding token
0            [PAD]           0            ← Padding token
0            [PAD]           0            ← Padding token
0            [PAD]           0            ← Padding token
0            [PAD]           0            ← Padding token
0            [PAD]           0            ← Padding token
0            [PAD]           0            ← Padding token


In [16]:
# 💡 Understanding Special Tokens

print("🏷️ Special Tokens in BERT Tokenizer:\n")

special_tokens = {
    '[CLS]': 'Classification token - marks the beginning of input',
    '[SEP]': 'Separator token - marks end of sentence or separates sentence pairs',
    '[PAD]': 'Padding token - fills sequences to uniform length',
    '[UNK]': 'Unknown token - represents out-of-vocabulary tokens',
    '[MASK]': 'Mask token - used for masked language modeling'
}

for token, description in special_tokens.items():
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"{token} (ID: {token_id})")
    print(f"   └── {description}\n")

🏷️ Special Tokens in BERT Tokenizer:

[CLS] (ID: 101)
   └── Classification token - marks the beginning of input

[SEP] (ID: 102)
   └── Separator token - marks end of sentence or separates sentence pairs

[PAD] (ID: 0)
   └── Padding token - fills sequences to uniform length

[UNK] (ID: 100)
   └── Unknown token - represents out-of-vocabulary tokens

[MASK] (ID: 103)
   └── Mask token - used for masked language modeling



### 🧠 Understanding Attention Mask

The attention mask tells the model:
- **1** = "Pay attention to this token" (real content)
- **0** = "Ignore this token" (padding)

This is crucial for:
- Batch processing with variable-length sequences
- Preventing the model from learning from padding tokens

In [17]:
# 📊 Visualizing Attention Mask

short_text = "AI is amazing"
long_text = "Artificial intelligence transforms how we interact with technology daily"

# Tokenize both with same max_length
short_encoded = tokenizer(short_text, padding='max_length', max_length=20, truncation=True)
long_encoded = tokenizer(long_text, padding='max_length', max_length=20, truncation=True)

print("📊 Attention Mask Comparison:\n")

print(f"Short text: '{short_text}'")
print(f"Tokens:     {tokenizer.convert_ids_to_tokens(short_encoded['input_ids'])}")
print(f"Attention:  {short_encoded['attention_mask']}")
print(f"Real tokens: {sum(short_encoded['attention_mask'])} | Padding: {len(short_encoded['attention_mask']) - sum(short_encoded['attention_mask'])}")

print(f"\nLong text: '{long_text}'")
print(f"Tokens:     {tokenizer.convert_ids_to_tokens(long_encoded['input_ids'])}")
print(f"Attention:  {long_encoded['attention_mask']}")
print(f"Real tokens: {sum(long_encoded['attention_mask'])} | Padding: {len(long_encoded['attention_mask']) - sum(long_encoded['attention_mask'])}")

📊 Attention Mask Comparison:

Short text: 'AI is amazing'
Tokens:     ['[CLS]', 'ai', 'is', 'amazing', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention:  [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Real tokens: 5 | Padding: 15

Long text: 'Artificial intelligence transforms how we interact with technology daily'
Tokens:     ['[CLS]', 'artificial', 'intelligence', 'transforms', 'how', 'we', 'interact', 'with', 'technology', 'daily', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
Attention:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Real tokens: 11 | Padding: 9


### 🔗 Understanding Token Type IDs

Token type IDs are used when the input consists of **multiple segments** (e.g., question-answering tasks):
- **0** = First segment (e.g., question)
- **1** = Second segment (e.g., context/answer)

For single-sentence inputs, all token type IDs are 0.

In [18]:
# 🔗 Token Type IDs with Sentence Pairs

question = "What is machine learning?"
answer = "Machine learning is a subset of AI that enables computers to learn from data."

# Tokenize as a sentence pair
pair_encoded = tokenizer(
    question, 
    answer, 
    padding='max_length', 
    max_length=35,
    truncation=True
)

tokens = tokenizer.convert_ids_to_tokens(pair_encoded['input_ids'])

print("🔗 Sentence Pair Tokenization:\n")
print(f"Question: '{question}'")
print(f"Answer: '{answer}'\n")

print(f"{'Token':<20} {'Type ID':<10} {'Segment'}")
print("-" * 45)

for token, type_id in zip(tokens, pair_encoded['token_type_ids']):
    if token == '[PAD]':
        continue
    segment = "Question" if type_id == 0 else "Answer"
    if token in ['[CLS]', '[SEP]']:
        segment = f"Special ({segment})"
    print(f"{token:<20} {type_id:<10} {segment}")

🔗 Sentence Pair Tokenization:

Question: 'What is machine learning?'
Answer: 'Machine learning is a subset of AI that enables computers to learn from data.'

Token                Type ID    Segment
---------------------------------------------
[CLS]                0          Special (Question)
what                 0          Question
is                   0          Question
machine              0          Question
learning             0          Question
?                    0          Question
[SEP]                0          Special (Question)
machine              1          Answer
learning             1          Answer
is                   1          Answer
a                    1          Answer
subset               1          Answer
of                   1          Answer
ai                   1          Answer
that                 1          Answer
enables              1          Answer
computers            1          Answer
to                   1          Answer
learn               

---

## 📖 Part 4: Tokenizing Datasets for Training

Let's apply everything we've learned to tokenize a real dataset, preparing it for model training!

In [19]:
# 📥 Load a Sample Dataset

# Using the 'emotion' dataset - classifying text into emotions
dataset = load_dataset('dair-ai/emotion', split='train[:1000]')  # First 1000 samples

print("📊 Dataset Info:")
print(f"   • Number of samples: {len(dataset)}")
print(f"   • Features: {dataset.features}")
print(f"\n📝 Sample entries:")
for i in range(3):
    print(f"   {i+1}. '{dataset[i]['text'][:60]}...' → Label: {dataset[i]['label']}")

README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

📊 Dataset Info:
   • Number of samples: 1000
   • Features: {'text': Value('string'), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])}

📝 Sample entries:
   1. 'i didnt feel humiliated...' → Label: 0
   2. 'i can go from feeling so hopeless to so damned hopeful just ...' → Label: 0
   3. 'im grabbing a minute to post i feel greedy wrong...' → Label: 3


In [20]:
# 🔧 Define Tokenization Function

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    """Tokenize a batch of examples."""
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

print("✅ Tokenization function defined!")
print("\n📋 Parameters:")
print("   • truncation=True: Cut sequences longer than max_length")
print("   • padding='max_length': Pad shorter sequences")
print("   • max_length=128: Maximum sequence length")

✅ Tokenization function defined!

📋 Parameters:
   • truncation=True: Cut sequences longer than max_length
   • padding='max_length': Pad shorter sequences
   • max_length=128: Maximum sequence length


In [21]:
# ⚡ Apply Tokenization to Dataset

# Using the map() function for efficient batch processing
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,           # Process in batches for efficiency
    remove_columns=['text'] # Remove original text column
)

print("✅ Dataset tokenized successfully!")
print(f"\n📊 Tokenized Dataset Features:")
print(f"   {tokenized_dataset.features}")
print(f"\n📋 New columns added:")
print("   • input_ids: Token IDs")
print("   • attention_mask: Real token indicators")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset tokenized successfully!

📊 Tokenized Dataset Features:
   {'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']), 'input_ids': List(Value('int32')), 'attention_mask': List(Value('int8'))}

📋 New columns added:
   • input_ids: Token IDs
   • attention_mask: Real token indicators


In [22]:
# 🔍 Examine a Tokenized Sample

sample_idx = 0
sample = tokenized_dataset[sample_idx]

print(f"📋 Tokenized Sample #{sample_idx}:\n")
print(f"Label: {sample['label']}")
print(f"\nInput IDs (first 30): {sample['input_ids'][:30]}")
print(f"Attention Mask (first 30): {sample['attention_mask'][:30]}")

# Decode back to text
decoded_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
print(f"\n🔄 Decoded text: '{decoded_text}'")

📋 Tokenized Sample #0:

Label: 0

Input IDs (first 30): [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention Mask (first 30): [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

🔄 Decoded text: 'i didnt feel humiliated'


In [23]:
# 📊 Analyze Token Distribution

# Count actual tokens (excluding padding)
token_counts = [sum(sample['attention_mask']) for sample in tokenized_dataset]

avg_tokens = sum(token_counts) / len(token_counts)
max_tokens = max(token_counts)
min_tokens = min(token_counts)

print("📊 Token Distribution Statistics:\n")
print(f"   • Average tokens per sample: {avg_tokens:.1f}")
print(f"   • Maximum tokens: {max_tokens}")
print(f"   • Minimum tokens: {min_tokens}")
print(f"   • Max length setting: 128")
print(f"\n💡 Insight: {'Most samples use full length!' if avg_tokens > 100 else 'Samples are shorter than max_length - consider reducing max_length for efficiency.'}") 

📊 Token Distribution Statistics:

   • Average tokens per sample: 22.9
   • Maximum tokens: 70
   • Minimum tokens: 5
   • Max length setting: 128

💡 Insight: Samples are shorter than max_length - consider reducing max_length for efficiency.


---

## 📖 Part 5: Advanced Tokenization Techniques

In [24]:
# 🎯 Dynamic Padding (More Efficient)

# Instead of padding to max_length, pad to longest in batch
texts = [
    "Short text",
    "This is a medium length sentence",
    "This is a much longer sentence that contains more words and information"
]

# With dynamic padding
dynamic_encoded = tokenizer(
    texts,
    padding=True,  # Pad to longest in batch
    truncation=True,
    return_tensors='pt'  # Return PyTorch tensors
)

print("🎯 Dynamic Padding Results:\n")
print(f"Shape of input_ids: {dynamic_encoded['input_ids'].shape}")
print(f"(batch_size=3, sequence_length={dynamic_encoded['input_ids'].shape[1]})")
print("\n💡 Sequence length is determined by the longest text in the batch!")

🎯 Dynamic Padding Results:

Shape of input_ids: torch.Size([3, 14])
(batch_size=3, sequence_length=14)

💡 Sequence length is determined by the longest text in the batch!


In [27]:
# 🔀 Handling Multiple Text Pairs

# Useful for NLI, QA, and sentence similarity tasks
premises = [
    "The weather is sunny today",
    "Python is a programming language"
]

hypotheses = [
    "It's a beautiful day outside",
    "Python is used for web development"
]

# Use BERT tokenizer for this example (it returns token_type_ids)
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

pair_encoded = bert_tokenizer(
    premises,
    hypotheses,
    padding=True,
    truncation=True,
    return_tensors='pt'
)

print("🔀 Batch Sentence Pair Encoding:\n")
print(f"Input IDs shape: {pair_encoded['input_ids'].shape}")
print(f"   → (batch_size={pair_encoded['input_ids'].shape[0]}, sequence_length={pair_encoded['input_ids'].shape[1]})")

# Show first pair details
print(f"\n📝 First Pair:")
print(f"   Premise: '{premises[0]}'")
print(f"   Hypothesis: '{hypotheses[0]}'")
print(f"\n   Tokens: {bert_tokenizer.convert_ids_to_tokens(pair_encoded['input_ids'][0].tolist())}")

# Show token_type_ids to distinguish premise from hypothesis
print(f"\n   Token Type IDs: {pair_encoded['token_type_ids'][0].tolist()}")
print("   → 0 = Premise tokens, 1 = Hypothesis tokens")

# Show second pair for comparison
print(f"\n📝 Second Pair:")
print(f"   Premise: '{premises[1]}'")
print(f"   Hypothesis: '{hypotheses[1]}'")
print(f"\n   Tokens: {bert_tokenizer.convert_ids_to_tokens(pair_encoded['input_ids'][1].tolist())}")

🔀 Batch Sentence Pair Encoding:

Input IDs shape: torch.Size([2, 15])
   → (batch_size=2, sequence_length=15)

📝 First Pair:
   Premise: 'The weather is sunny today'
   Hypothesis: 'It's a beautiful day outside'

   Tokens: ['[CLS]', 'the', 'weather', 'is', 'sunny', 'today', '[SEP]', 'it', "'", 's', 'a', 'beautiful', 'day', 'outside', '[SEP]']

   Token Type IDs: [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
   → 0 = Premise tokens, 1 = Hypothesis tokens

📝 Second Pair:
   Premise: 'Python is a programming language'
   Hypothesis: 'Python is used for web development'

   Tokens: ['[CLS]', 'python', 'is', 'a', 'programming', 'language', '[SEP]', 'python', 'is', 'used', 'for', 'web', 'development', '[SEP]', '[PAD]']


---

## 🎓 Key Takeaways

### 1. Tokenization Types
- **Word-level**: Simple but limited vocabulary and OOV handling
- **Subword-level** (BPE, WordPiece): Best of both worlds - used by modern transformers
- **Character-level**: Good for CJK languages and typo handling

### 2. Tokenizer Outputs
- **`input_ids`**: Numerical token representations
- **`attention_mask`**: 1 for real tokens, 0 for padding
- **`token_type_ids`**: Segment identifiers for sentence pairs

### 3. Special Tokens
- **[CLS]**: Start of sequence (classification)
- **[SEP]**: Separator/End of sequence
- **[PAD]**: Padding for batch processing
- **[MASK]**: For masked language modeling

### 4. Best Practices
- Use `AutoTokenizer` for model-compatible tokenization
- Apply `truncation=True` and set appropriate `max_length`
- Use `batched=True` with `dataset.map()` for efficiency
- Consider dynamic padding for variable-length sequences

---

## 📚 Further Reading

- [Hugging Face Tokenizers Documentation](https://huggingface.co/docs/tokenizers/)
- [Understanding WordPiece](https://huggingface.co/learn/nlp-course/chapter6/6)
- [BPE Algorithm Explained](https://huggingface.co/learn/nlp-course/chapter6/5)

---